#CM4709 Computer Vision
#Lab 09 Time Series Prediction Using LSTM

Note: This lab is based on the [Youtube video here](https://youtu.be/c0k-YLQGKjY).

##Aims
* Use the Tensorflow LSTM library/model to predict a time series.

##Downloading Dataset

The [Jena climate dataset](https://keras.io/examples/timeseries/timeseries_weather_forecasting/) is a time series that consists of 14 features like temperature, pressure, humidity, etc.
We will use the dataset to predict temperature only.
The data set is huge, with data recorded every 10 minutes for 8 years from 2009 to 2016.

The following code (adapted from the Youtube video) download the dataset:

In [ ]:
import tensorflow as tf
import os

zip_path = tf.keras.utils.get_file(
    origin='https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip',
    fname='jena_climate_2009_2016.csv.zip',
    extract=True)
csv_path, _ = os.path.splitext(zip_path)
print(csv_path)

Alternatively, you can use the following shell commands to download and unzip the dataset.
The path to the CSV file will be different, but it is fine as far as the `csv_path` value is set correctly:

In [ ]:
!wget https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip
!unzip jena_climate_2009_2016.csv.zip
csv_path='jena_climate_2009_2016.csv'

##Exploring the Dataset

The following codes read the CSV file into a dataframe and dump out the data:

In [ ]:
import pandas as pd

df_original=pd.read_csv(csv_path)
df_original

The dataset is huge, as data were logged with 10 minutes for 8 years.
We do not need such a high resolution sampling, and will only take data at every hour.
The following code will get data from the original dataframe from row 5, and every 6 rows. This will be the data at every hour:


In [ ]:
df=df_original[5::6]
df

We will now set the dataframe index to be the timestamp, with a different format:

In [ ]:
df.index = pd.to_datetime(df['Date Time'], format='%d.%m.%Y %H:%M:%S')
df.head(10)

To get the temperature column and plot it:

In [ ]:
temp = df['T (degC)']
temp.plot()

##Prepare Input and Label

To predict the temperature at time t, we will use the 5 previous temperature data at time t-5, t-4, t-3, t-2 and t-1.
We need to reshape our temperature column value into the correct shape.

In [ ]:
import numpy as np

# [[[1], [2], [3], [4], [5]]] [6]
# [[[2], [3], [4], [5], [6]]] [7]
# [[[3], [4], [5], [6], [7]]] [8]

def df_to_x_y(df, window_size=5):
  df_as_np = df.to_numpy()
  x = []
  y = []
  for i in range(len(df_as_np)-window_size):
    row = [[a] for a in df_as_np[i:i+window_size]]
    x.append(row)
    label = df_as_np[i+window_size]
    y.append(label)
  return np.array(x), np.array(y)

In [ ]:
WINDOW_SIZE = 5
x1, y1 = df_to_x_y(temp, WINDOW_SIZE)
print('x shape: ',x1.shape)
print('y shape: ',y1.shape)
print('===X===')
print(x1[:5])
print('===Y===')
print(y1[:5])

##Training, Validation, and Testing Datasets

We will split the dataset into training, validation, and testing.

In [ ]:
#take 1st 60000 entries as training data
x_train1, y_train1 = x1[:60000], y1[:60000]

#next 5000 entries as validation data
x_val1, y_val1 = x1[60000:65000], y1[60000:65000]

#the rest as testing data
x_test1, y_test1 = x1[65000:], y1[65000:]

#print shapes of all subsets
print('x_train shape: ',x_train1.shape)
print('y_train shape: ', y_train1.shape)
print('x_val shape: ',x_val1.shape)
print('y_val shape: ',y_val1.shape)
print('x_test shape:',x_test1.shape)
print('y_test shape:', y_test1.shape)

##Creating the LSTM Model

We can now create the LSTM model.
The process is similar to how we create other NN.
Note the use of the [`LSTM`](https://keras.io/api/layers/recurrent_layers/lstm/) layer.
Notice the shape of the input and output layers.

Also note that LSTM is keeping memory of the whole sequence. So the output is not only dependent on the input values at an instance, but all previous inputs.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

model1 = Sequential()
model1.add(InputLayer((5, 1)))
model1.add(LSTM(64))
model1.add(Dense(8, 'relu'))
model1.add(Dense(1, 'linear'))

model1.summary()

##Compile & Train Model

Training of the model is quite standard.
We add a callback to create a checkpoint that saves the best model during validation.

In [ ]:
#callback for saving best model during training
cp1 = ModelCheckpoint('best_model.keras', save_best_only=True)

#compile model
model1.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])

#train model
model1.fit(x_train1, y_train1, validation_data=(x_val1, y_val1), epochs=10, callbacks=[cp1])

##Prediction vs Actual in Training Dataset

In most cases we won't do this as the model is trained using the training data. It should perform well on this.

In [ ]:
from tensorflow.keras.models import load_model

#load the best model
model1 = load_model('best_model.keras')

In [ ]:
#do predict on training data
train_predictions = model1.predict(x_train1).flatten()

#print prediction vs actual
train_results = pd.DataFrame(data={'Train Predictions':train_predictions, 'Actuals':y_train1})
train_results

##Plot Prediction vs Actual in Training Dataset

In [ ]:
import matplotlib.pyplot as plt
plt.plot(train_results['Train Predictions'][:100])
plt.plot(train_results['Actuals'][:100])


##Prediction vs Actual in Validation Dataset

In [ ]:
val_predictions = model1.predict(x_val1).flatten()
val_results = pd.DataFrame(data={'Val Predictions':val_predictions, 'Actuals':y_val1})
val_results

In [ ]:
plt.plot(val_results['Val Predictions'][:100])
plt.plot(val_results['Actuals'][:100])

##Prediction vs Actual in Testing Dataset

In [ ]:
test_predictions = model1.predict(x_test1).flatten()
test_results = pd.DataFrame(data={'Test Predictions':test_predictions, 'Actuals':y_test1})
test_results

In [ ]:
plt.plot(test_results['Test Predictions'][:100])
plt.plot(test_results['Actuals'][:100])

##Other Prediction Techniques

If you are interested to go further, there is a [Part 2 of the Youtube video](https://youtu.be/kGdbPnMCdOg) which uses other techniques like GRU and 1d CNN.